# FOOD CLASSIFICATION PROJECT

*Pytorch project for AI Engineering Master with ProfessionAI*

## Colab Configuration

If you're testing this project on Google Colab, it could be useful for you to run the following cells

In [ ]:
# If you are running this code on Google Colab, run this:

!git clone https://github.com/Silvano315/PyTorch-CNN-for-food-image-classification-system.git

%cd PyTorch-CNN-for-food-image-classification-system
%pwd

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

## Import Libraries and Datasets

In [4]:
# Import libraries

import os
import sys
import json
import random

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchsummary import summary
import albumentations as A
from albumentations.pytorch import ToTensorV2

from src.constants import RANDOM_SEED

In [5]:
# Set Random Seed for reproducibility

os.environ['PYTHONHASHSEED'] = str(RANDOM_SEED)
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

## Exploratory Data Analysis